In [1]:
from pathlib import Path
import sys

root_dir = Path().resolve().parent.parent.parent.as_posix()
if root_dir not in sys.path:
    sys.path.append(root_dir)
del root_dir

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

from utils import config, datasets, models

In [2]:
MODEL_NAME = 'no_base_model-functional'
EPOCHS = 200

In [3]:
train_dataset, validation_dataset, test_dataset = datasets.load_dataset()

Found 6400 files belonging to 4 classes.


In [4]:
preprocessing = keras.Sequential([
    layers.Rescaling(1. / 255),
])

In [5]:
input_ = keras.layers.Input(shape=tf.TensorShape([128, 128, 3]), batch_size=32)

conv1_1 = layers.Conv2D(filters=32, kernel_size=(1, 1), padding='same',
                        activation=keras.activations.relu)(input_)
conv1_3 = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same',
                        activation=keras.activations.relu)(input_)
conv1_5 = layers.Conv2D(filters=32, kernel_size=(5, 5), padding='same',
                        activation=keras.activations.relu)(input_)

concatenate_1 = layers.Concatenate()([conv1_1, conv1_3, conv1_5])
pool_1 = layers.MaxPooling2D()(concatenate_1)

conv2_3 = layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same',
                        activation=keras.activations.relu)(pool_1)
conv2_5 = layers.Conv2D(filters=64, kernel_size=(5, 5), padding='same',
                        activation=keras.activations.relu)(pool_1)

concatenate_2 = layers.Concatenate()([conv2_3, conv2_5])
pool_2 = layers.MaxPooling2D()(concatenate_2)

dropout_2 = layers.Dropout(0.2)(pool_2)

conv3_3 = layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same',
                        activation=keras.activations.relu)(dropout_2)

pool_3 = layers.MaxPooling2D()(conv3_3)
dropout_3 = layers.Dropout(0.25)(pool_3)

flatten = layers.Flatten()(dropout_3)

dense_1 = layers.Dense(256, activation=keras.activations.relu,
                       kernel_regularizer=keras.regularizers.l2(0.001))(flatten)
dense_2 = layers.Dense(256, activation=keras.activations.relu,
                       kernel_regularizer=keras.regularizers.l2(0.001))(dense_1)
output = layers.Dense(4, activation=keras.activations.softmax)(dense_2)

top = keras.Model(inputs=input_, outputs=output)

In [6]:
model = models.create_model(
    model_name=MODEL_NAME,
    preprocessing_layers=preprocessing,
    base_model=None,
    top_layers=top,
    optimizer=keras.optimizers.Adam()
)

In [7]:
history = models.fit_model(
    model,
    train_data=train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS
)

Epoch 1/200
160/160 [==============================] - 15s 77ms/step - loss: 1.3644 - sparse_categorical_accuracy: 0.4932 - val_loss: 1.1183 - val_sparse_categorical_accuracy: 0.5156
Epoch 2/200
160/160 [==============================] - 12s 75ms/step - loss: 1.0436 - sparse_categorical_accuracy: 0.5691 - val_loss: 1.0190 - val_sparse_categorical_accuracy: 0.5406
Epoch 3/200
160/160 [==============================] - 12s 75ms/step - loss: 0.9655 - sparse_categorical_accuracy: 0.5945 - val_loss: 0.9240 - val_sparse_categorical_accuracy: 0.6172
Epoch 4/200
160/160 [==============================] - 12s 75ms/step - loss: 0.8964 - sparse_categorical_accuracy: 0.6314 - val_loss: 0.8828 - val_sparse_categorical_accuracy: 0.6578
Epoch 5/200
160/160 [==============================] - 12s 75ms/step - loss: 0.8143 - sparse_categorical_accuracy: 0.6785 - val_loss: 0.7779 - val_sparse_categorical_accuracy: 0.7078
Epoch 6/200
160/160 [==============================] - 12s 75ms/step - loss: 0.7067 -

In [8]:
model.summary()

Model: "no_base_model-functional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 128, 128, 3)       0         
                                                                 
 model (Functional)          (32, 4)                   8815748   
                                                                 
Total params: 8,815,748
Trainable params: 8,815,748
Non-trainable params: 0
_________________________________________________________________
